<div style="background-color: #e8f4f8; border: 2px solid #b6d4e0; padding: 20px; border-radius: 10px; margin: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h4 style="color: #2c7be5; font-size: 22px; font-family: 'Verdana', sans-serif; margin-bottom: 15px;">Requirements</h4>
    
</div>

In [ ]:
!pip install transformers spacy torch sentencepiece

In [ ]:
!pip install protobuf==3.20.3


In [ ]:
!python -m spacy download fr_core_news_lg

<div style="background-color: #e8f4f8; border: 2px solid #b6d4e0; padding: 20px; border-radius: 10px; margin: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h4 style="color: #2c7be5; font-size: 22px; font-family: 'Verdana', sans-serif; margin-bottom: 15px;">Data import</h4>
    
</div>

In [ ]:
data_path='dataset.txt'

<div style="background-color: #e8f4f8; border: 2px solid #b6d4e0; padding: 20px; border-radius: 10px; margin: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h4 style="color: #2c7be5; font-size: 22px; font-family: 'Verdana', sans-serif; margin-bottom: 15px;">Spacy NER</h4>
    
</div>

In [ ]:
import spacy
import re

nlp =spacy.load("fr_core_news_lg")

with open(data_path, "r", encoding="utf-8", errors='ignore') as file:
    # Read the contents of the file
    text = file.read()
text_without_brackets = re.sub(r'\[[^\]]*\]', '', text)
# Print the contents of the file


doc = nlp(text_without_brackets) 

spacy.displacy.render(doc, style="ent", jupyter=True) 

<div style="background-color: #e8f4f8; border: 2px solid #b6d4e0; padding: 20px; border-radius: 10px; margin: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h4 style="color: #2c7be5; font-size: 22px; font-family: 'Verdana', sans-serif; margin-bottom: 15px;">Camembert NER</h4>
    
</div>

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)




In [ ]:
def clean_concat_tokens(tokens, is_date=False, last_token_was_numeric=False):
    """Cleans and concatenates a list of tokens."""
    new_tokens = []
    for i, token in enumerate(tokens):
        if is_date and token.isnumeric() and (i == 0 or last_token_was_numeric):
            # No space before numeric tokens if they are part of a date
            new_tokens.append(token)
        else:
            # Add a space before tokens that are not at the beginning or not numeric
            if i > 0:
                new_tokens.append(" ")
            new_tokens.append(token)
        last_token_was_numeric = token.isnumeric()
    return "".join(new_tokens)


def identify_and_group_entities(text):
    global i
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)
    tokens = inputs["input_ids"]

    current_entity_tokens = []
    current_entity_type = None
    last_token_was_numeric = False
    entity_start = None

    for token, prediction in zip(tokens[0], predictions[0]):
        token_text = tokenizer.decode([token], skip_special_tokens=True).strip()
        entity = model.config.id2label[prediction.item()]
        if entity != "O":
            if current_entity_tokens and (entity != current_entity_type or entity.startswith('B')):
                # Print the previous entity and start a new one
                entity_end = i - 1
                print(f"{clean_concat_tokens(current_entity_tokens, is_date='DATE' in current_entity_type, last_token_was_numeric=last_token_was_numeric)}, Entity: {current_entity_type}, Start: {entity_start}, End: {entity_end}\n")
                current_entity_tokens = [token_text]
                last_token_was_numeric = token_text.isnumeric()
                entity_start = i
            else:
                # Accumulate tokens for the same entity or logical sequence
                current_entity_tokens.append(token_text)
                last_token_was_numeric = token_text.isnumeric()
                if entity_start is None:
                    entity_start = i
            current_entity_type = entity
        else:
            if current_entity_tokens:
                # At the end of an entity, print and reset
                entity_end = i - 1
                print(f"{clean_concat_tokens(current_entity_tokens, is_date='DATE' in current_entity_type, last_token_was_numeric=last_token_was_numeric)}, Entity: {current_entity_type}, Start: {entity_start}, End: {entity_end}\n")
                current_entity_tokens = []
                last_token_was_numeric = False
                entity_start = None
            current_entity_type = None
        i += 1
    # Make sure to print the last entity if necessary
    if current_entity_tokens:
        entity_end = len(tokens[0]) - 1
        print(f"{clean_concat_tokens(current_entity_tokens, is_date='DATE' in current_entity_type, last_token_was_numeric=last_token_was_numeric)}, Entity: {current_entity_type}, Start: {entity_start}, End: {entity_end}\n")

i = 0
# Read the file and extract entities
with open(data_path, "r", encoding="utf-8") as file:
    for line in file:
        identify_and_group_entities(line.strip())

, Entity: I-PER, Start: 0, End: 0

Henri IV, Entity: I-PER, Start: 8, End: 9

Henri de Bourbon, Entity: I-PER, Start: 11, End: 13

Henri III, Entity: I-PER, Start: 15, End: 16

Henri de Navarre, Entity: I-PER, Start: 18, End: 20

Henri de France, Entity: I-PER, Start: 22, End: 24

, Entity: I-PER, Start: 26, End: 26

Le Vert Gala nt, Entity: I-LOC, Start: 29, End: 32

Le Vert Gala nt, Entity: I-LOC, Start: 46, End: 49

Henri IV, Entity: I-PER, Start: 57, End: 58

« le Grand, Entity: I-PER, Start: 61, End: 63

Le Vert Gala nt, Entity: I-PER, Start: 67, End: 70

d ' Henri de Bourbon, Entity: I-PER, Start: 76, End: 80

le 13 décembre 1553 à, Entity: I-DATE, Start: 81, End: 86

Pau, Entity: I-LOC, Start: 87, End: 87

le 14 mai 1610 à, Entity: I-DATE, Start: 91, End: 96

Paris, Entity: I-LOC, Start: 97, End: 97

de Navarre, Entity: I-LOC, Start: 101, End: 102

9 juin 1572, Entity: I-DATE, Start: 106, End: 109

d ' Henri III, Entity: I-PER, Start: 113, End: 116

roi, Entity: I-PER, Start: 11

<div style="background-color: #e8f4f8; border: 2px solid #b6d4e0; padding: 20px; border-radius: 10px; margin: 20px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h4 style="color: #2c7be5; font-size: 22px; font-family: 'Verdana', sans-serif; margin-bottom: 15px;">Spacy and Camembert NER</h4>
    
</div>

In [ ]:
import spacy
import re

nlp =spacy.load("fr_core_news_lg")

with open(data_path, "r", encoding="utf-8", errors='ignore') as file:
    # Read the contents of the file
    text = file.read()
text_without_brackets = re.sub(r'\[[^\]]*\]', '', text)
# Print the contents of the file


doc = nlp(text_without_brackets) 



In [ ]:
def clean_concat_tokens(tokens, is_date=False, last_token_was_numeric=False):
    """Cleans and concatenates a list of tokens."""
    new_tokens = []
    for i, token in enumerate(tokens):
        if is_date and token.isnumeric() and (i == 0 or last_token_was_numeric):
            # No space before numeric tokens if they are part of a date
            new_tokens.append(token)
        else:
            # Add a space before tokens that are not at the beginning or not numeric
            if i > 0:
                new_tokens.append(" ")
            new_tokens.append(token)
        last_token_was_numeric = token.isnumeric()
    return "".join(new_tokens)

def identify_and_group_entities(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)
    tokens = inputs["input_ids"]
    spacy = doc.ents
    p = 0
    current_entity_tokens = []
    current_entity_type = None
    last_token_was_numeric = False

    for token, prediction in zip(tokens[0], predictions[0]):
        token_text = tokenizer.decode([token], skip_special_tokens=True).strip()
        entity = model.config.id2label[prediction.item()]
        if entity != "O":
            if current_entity_tokens and (entity != current_entity_type or entity.startswith('B')):
                if current_entity_type != 'I-PER' and current_entity_type != 'I-LOC':
                    # Print the previous entity and start a new one
                    print(f"{clean_concat_tokens(current_entity_tokens)}, Entity: {current_entity_type}\n")
                    current_entity_tokens = [token_text]
                    current_entity_type = entity
                else:
                    # Print the spaCy entity for PER and LOC
                    print(f"Text: {spacy[p].text}, Label: {spacy[p].label_}, Start: {spacy[p].start}, End: {spacy[p].end}\n")
                    p += 1
                    current_entity_tokens = [token_text]
                    current_entity_type = entity
            else:
                # Accumulate tokens for the same entity or logical sequence
                current_entity_tokens.append(token_text)
                last_token_was_numeric = token_text.isnumeric()
            current_entity_type = entity
        else:
            if current_entity_tokens:
                if current_entity_type != 'I-PER' and current_entity_type != 'I-LOC':
                    # Print the entity if not PER or LOC
                    print(f"{clean_concat_tokens(current_entity_tokens, is_date='DATE' in current_entity_type, last_token_was_numeric=last_token_was_numeric)}, Entity: {current_entity_type}\n")
                    current_entity_tokens = []
                    last_token_was_numeric = False
                else:
                    # Print the spaCy entity for PER and LOC
                    print(f"Text: {spacy[p].text}, Label: {spacy[p].label_}, Start: {spacy[p].start}, End: {spacy[p].end}\n")
                    p += 1
                    current_entity_tokens = []
                    last_token_was_numeric = False
                # At the end of an entity, print and reset
                current_entity_type = None

    # Make sure to print the last entity if necessary
    if current_entity_tokens:
        print(f"{clean_concat_tokens(current_entity_tokens, is_date='DATE' in current_entity_type, last_token_was_numeric=last_token_was_numeric)}, Entity: {current_entity_type}")

# Read the file and extract entities
with open(data_path, "r", encoding="utf-8") as file:
    for line in file:
        identify_and_group_entities(line.strip())


Text: Henri IV, Label: PER, Start: 6, End: 9

Text: Henri de Bourbon, Label: PER, Start: 10, End: 13

Text: Henri III, Label: PER, Start: 14, End: 17

Text: Henri de Navarre, Label: PER, Start: 18, End: 21

Text: Henri de France, Label: PER, Start: 22, End: 25

Text: Le Vert Galant, Label: MISC, Start: 29, End: 32

, Entity: I-PER
Text: Henri IV, Label: PER, Start: 6, End: 9

Text: Henri de Bourbon, Label: PER, Start: 10, End: 13

Text: Henri IV, Label: PER, Start: 6, End: 9

Text: Henri de Bourbon, Label: PER, Start: 10, End: 13

Text: Henri III, Label: PER, Start: 14, End: 17

Text: Henri de Navarre, Label: PER, Start: 18, End: 21

le 13 décembre 15 53 à, Entity: I-DATE

Text: Henri de France, Label: PER, Start: 22, End: 25

le 14 mai 16 10 à, Entity: I-DATE

Text: Le Vert Galant, Label: MISC, Start: 29, End: 32

Text: Le Vert Galant, Label: MISC, Start: 43, End: 46

9 juin 1572, Entity: I-DATE

Text: Henri IV, Label: PER, Start: 51, End: 54

Text: Grand, Label: LOC, Start: 59, End: 